In [1]:
%matplotlib nbagg
import cocoex
import numpy as np
import os
import sys
os.chdir('C:/Users/dimo/Desktop/code-mo-test/comocma/')
sys.path.insert(0, '.')
import como
from cocoex import default_observers 
from matplotlib import pyplot as plt
import functools


In [2]:
mysuite = cocoex.Suite('bbob-biobj', '', '')
dimension = 2
fun = mysuite.get_problem_by_function_dimension_instance(1, dimension, 3)
observer_name = default_observers()['bbob-biobj']
observer = cocoex.Observer(observer_name, '')
observer.observe(fun)
max_evals = 5e4 * dimension

In [ ]:
x0 = fun.initial_solution
fx0 = fun(x0)
num_kernels = 1  # number of single-objective solvers (population size)
sigma0 = 2

list_of_solvers = como.get_cmas(fun.dimension*[4], #-5 + 10 * np.random.rand(num_kernels, fun.dimension),
                                sigma0,  # inopts=cmaopts
                                {'tolx': 10 ** -4, 
                                 'tolfunrel': 1e-1,
                                 'maxiter': 1e9}
                                )  # produce `num_kernels cma instances`
my_random_restart_function = functools.partial(como.random_restart_kernel, 
                                            x0_fct=lambda n: n*[4],
                                            #opts={'seed':1000}
                                            )

moes = como.Sofomore(list_of_solvers, opts={'archive': False,
                                     #'restart': como.random_restart_kernel,
                                     'restart': my_random_restart_function,
                                     #'restart': como.best_chv_restart_kernel,
                                     #'restart': como.best_chv_or_random_restart_kernel,
                                     'continue_stopped_kernel': False,
                                     'random_restart_on_domination': False},
reference_point = fun.largest_fvalues_of_interest)  # create a como-cma-es instance
while fun.evaluations < max_evals:
    solutions = moes.ask()
    objective_values = [list(fun(x)) for x in solutions]
    moes.tell(solutions, objective_values)
    #print(str(fun.evaluations) + ' evaluations:') 
    moes.disp()
    #open('onlyrandom-countevals-3e4.tmp', 'w').write(repr([k.countevals for k in moes.kernels]))
    open('onlyrandom-countiters.tmp', 'w').write(repr([k.countiter for k in moes.kernels]))
    #open('onlyrandom-idsofrestarts.tmp', 'w').write(repr(moes._previous_kernel_ids_on_restart))
    #open('onlyrandom-bestchvsatrestart.tmp', 'w').write(repr(moes._bestchv_on_restart))
    #open('onlyrandom-seedsatrestart.tmp', 'w').write(repr(moes._seeds_at_restart))
    #open('onlyrandom-meansatrestart.tmp', 'w').write(repr(moes._meansAtRestart))
    

Iterat #Fevals   Hypervolume   axis ratios   sigmas   min&max stds
                                  (median)  (median)    (median)
    1      7 0.000000000000000e+00 1.0e+00 1.79e+00  2e+00  2e+00
    2     14 0.000000000000000e+00 1.1e+00 1.80e+00  1e+00  2e+00
    3     21 0.000000000000000e+00 1.3e+00 2.35e+00  2e+00  2e+00
  100    707 1.148027400386140e+02 1.4e+00 5.97e-01  2e-01  3e-01
  200   1417 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
  300   2127 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
  400   2837 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
  500   3547 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
  600   4257 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
  700   4967 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
  800   5677 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
  900   6387 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01
 1000   7097 1.149140357710794e+02 1.4e+00 9.35e-01  5e-01  5e-01


In [ ]:
plt.figure(76565564)
objvectors = np.array([k.objective_values for k in moes.kernels])
domobjvectors = np.array([k.objective_values for k in moes.kernels if k.objective_values not in moes.pareto_front])
plt.plot(objvectors[:,0], objvectors[:,1], 'xb')
if len(domobjvectors) > 1:
    plt.plot(domobjvectors[:,0], domobjvectors[:,1], 'xr')
#obj = np.array([moes.kernels[i].objective_values for i in range(len(hvc)) if hvc[i]<=1e-7])
#plt.plot(obj[:,0], obj[:,1], '.k')
#obj = np.array([moes.kernels[i].objective_values for i in range(len(hvc)) if hvc[i]==0])
#plt.plot(obj[:,0], obj[:,1], '.y')

plt.grid(which='both')
print(len(moes.kernels))

In [ ]:
objs = [tuple(k.objective_values) for k in moes]
print(len(objs))
print(len(set(objs)))

In [15]:
[k.objective_values for k in moes if k.objective_values not in moes.pareto_front]

[]

In [ ]:
len(set(objs))

In [25]:
plt.plot(np.array(moes.pareto_front)[:,0], np.array(moes.pareto_front)[:,1], '.r')